In [1]:
import json
import xml.etree.ElementTree as ET
import numpy as np
xml = ET.Element("NetSpeedData")
data=[]
with open('population_utf8.csv', 'r', encoding="utf-8") as pops, \
     open('results-2019-01-nopeustesti.csv', 'r', errors="ignore", encoding="utf-8") as nops, \
     open("netspeeds.json", "w") as jout, \
     open("netspeeds.xml", "w", encoding="utf-8") as xmlout:
         
    for line in pops:
        values = line.split(';')
        name = values[0].strip('\"')
        if len(values) == 5 : pop = values[2]
        elif len(values) == 6 : pop = values[3]
        data.append({
            'municipality': name,
            'population' : pop,
            'dls' : [],
            'uls' : []
            })
                
    for i, line in enumerate(nops):
        if i != 0:
            values = line.split(",")
            name = str(values[3])
            dls = str(values[4]) 
            uls = str(values[5])
            if name != "Joensuu":
                name = name.replace("ae","ä")
                name = name.replace("oe", "ö")
            for j in data:
                if j['municipality'] == name:
                    j['dls'].append(dls)
                    j['uls'].append(uls)
    
    for i in data:
        dlsum = 0
        ulsum = 0
        try : 
            dls = list(map(int, i["dls"]))
            uls = list(map(int, i["uls"]))
        except ValueError:
            pass 
        finally:
            ulcount = len(uls)
            dlcount = len(dls)
            if dlcount != 0: 
                for dl in dls:
                    dlsum += dl
                dlvar = np.var(dls)
                dlavg = dlsum/dlcount
                i.update({"DL avg": dlavg})
                i.update({"DL var": dlvar})
            else : 
                i.update({"DL avg": "n/a"})
                i.update({"DL var": "n/a"})
            if ulcount != 0: 
                for ul in uls:
                    ulsum += ul
                ulvar = np.var(uls)
                i.update({"UL avg": ulsum/ulcount})
                i.update({"UL var": ulvar})
            else :
                i.update({"UL avg": "n/a"})
                i.update({"UL var": "n/a"})
        
    for i in data:
        del i['dls']
        del i['uls']
        municipality = ET.SubElement(xml, "Municipality")
        name = ET.SubElement(municipality, "Name")
        name.text = i['municipality']
        population = ET.SubElement(municipality, "Population")
        population.text = i['population']
        dlavg = ET.SubElement(municipality, "DLavg")
        dlavg.text = str(i['DL avg'])
        dlvar = ET.SubElement(municipality, "DLvar")
        dlvar.text = str(i['DL var'])
        ulavg = ET.SubElement(municipality, "ULavg")
        ulavg.text = str(i['UL avg'])
        ulvar = ET.SubElement(municipality, "ULvar")
        ulvar.text = str(i['UL var'])
    
    json.dump(data, jout)
    mydata = ET.tostring(xml)
    mydata = mydata.decode('utf-8')
    xmlout.write(str(mydata))

FileNotFoundError: [Errno 2] No such file or directory: 'population_utf8.csv'